In [1]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
import os

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"] )

In [3]:
graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="12345678",
)

In [ ]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
For movie titles that begin with "The", move "the" to the end, For example "The 39 Steps" becomes "39 Steps, The" or "The Matrix" becomes "Matrix, The".

If no data is returned, do not attempt to answer the question.
Only respond to questions that require you to construct a Cypher statement.
Do not include any explanations or apologies in your responses.

Examples: 

Find movies and genres:
MATCH (m:Movie)-[:IN_GENRE]->(g)
RETURN m.title, g.name

Find roles for actors:
MATCH (m:Movie)-[r:ACTED_IN]->(p:Person)
WHERE m.title = 'movie title' AND p.name = 'actor name'
RETURN m.title, r.role, p.name

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

In [6]:
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True
)

cypher_chain.invoke({"query": "What movies has Tom Hanks directed and what are the genres?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Person {name: "Tom Hanks"})-[:DIRECTED]->(m:Movie)
MATCH (m)-[:IN_GENRE]->(g:Genre)
RETURN m.title AS Movie, g.name AS Genre
Full Context:
[{'Movie': 'That Thing You Do!', 'Genre': 'Drama'}, {'Movie': 'That Thing You Do!', 'Genre': 'Comedy'}, {'Movie': 'Larry Crowne', 'Genre': 'Drama'}, {'Movie': 'Larry Crowne', 'Genre': 'Romance'}, {'Movie': 'Larry Crowne', 'Genre': 'Comedy'}]

> Finished chain.


{'query': 'What movies has Tom Hanks directed and what are the genres?',
 'result': 'Tom Hanks has directed "That Thing You Do!" which falls under the genres Drama and Comedy, as well as "Larry Crowne" which is classified as Drama, Romance, and Comedy.'}